In [1]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv('data/Data4Good_Arolsen_Archives_50k.csv')

df.drop(['Unnamed: 0', 'TD', 'Last_Name', 'First Name', 'Alternative Name', 'Father (Vater - Eltern)', 'Mother (Mutter - Eltern)', 'Spouse (Ehem/Ehefr)', 'Alternative_Birthdate','Upper', 'Middle', 'Overall Confidence OCR', "Volunteers' Comment"], axis=1, inplace=True)

In [23]:
# Replace '//' with NaN
df['Birthdate (Geb)'] = df['Birthdate (Geb)'].replace('//', np.nan)

# Replace 'nan' with NaN
df['Birthdate (Geb)'] = df['Birthdate (Geb)'].replace('nan', np.nan)

# Convert to string to avoid errors
df['Birthdate (Geb)'] = df['Birthdate (Geb)'].astype(str)

# Function to extract the birth year
def extract_birth_year(date):
    if pd.isna(date) or date.strip() == '':
        return None  # Handle missing values
    parts = date.split('/')
    return parts[-1] if parts[-1].isdigit() and len(parts[-1]) == 4 else None  # Extract the last part if it's a valid year

# Apply the function to extract birth year
df['Birth_Year'] = df['Birthdate (Geb)'].apply(extract_birth_year).astype('Int64')  # Convert to integer, allowing NaN

# Drop the original columns (optional)
df.drop(columns=['Birthdate (Geb)'], inplace=True)


In [24]:
df.loc[df['Birth_Year'] < 1845, 'Birth_Year'] = np.nan
df.loc[df['Birth_Year'] > 1945, 'Birth_Year'] = np.nan


# Verify the changes
print(df['Birth_Year'].value_counts())


Birth_Year
1924    2198
1923    2175
1922    2139
1925    2119
1920    1792
        ... 
1855       5
1854       4
1852       4
1853       3
1849       2
Name: count, Length: 97, dtype: Int64


RELIGION

In [25]:
# Group rare religions
min_count = 5  # Threshold for rare categories
religion_counts = df['Religion'].value_counts()
rare_religions = religion_counts[religion_counts < min_count].index
df['Religion'] = df['Religion'].replace(rare_religions, 'Other')

In [26]:
df['Religion'].value_counts()

Religion
Jewish                   10886
Roman Catholic            1340
Other                      168
Orthodox Christian          73
Evangelical Christian       24
Name: count, dtype: int64

Birth Place

In [27]:
min_count = 100  # Threshold for rare categories
birth_place_counts = df['Birth Place'].value_counts()
rare_birth_places = birth_place_counts[birth_place_counts < min_count].index
df['Birth Place'] = df['Birth Place'].replace(rare_birth_places, 'Other')

In [28]:
df["Nationality"] = df["Nationality"].str.replace(r"(?i)\bFormerly\b", "", regex=True).str.strip()

df["Nationality"] = df["Nationality"].str.lower().replace(r"stateless", np.nan, regex=True)

In [29]:
from fuzzywuzzy import process
import pandas as pd

# Convert valid nationalities list to title case (First letter uppercase)
valid_nationalities = [
    # Common nationalities (ISO 3166)
    "Afghan", "Albanian", "Algerian", "American", "Andorran", "Angolan", "Argentine", "Armenian", "Australian", "Austrian",
    "Azerbaijani", "Bahamian", "Bahraini", "Bangladeshi", "Barbadian", "Belarusian", "Belgian", "Belizean", "Beninese", "Bhutanese",
    "Bolivian", "Bosnian", "Botswanan", "Brazilian", "British", "Bruneian", "Bulgarian", "Burkinabé", "Burmese", "Burundian",
    "Cambodian", "Cameroonian", "Canadian", "Cape Verdean", "Central African", "Chadian", "Chilean", "Chinese", "Colombian",
    "Comoran", "Congolese", "Costa Rican", "Croatian", "Cuban", "Cypriot", "Czech", "Danish", "Djiboutian", "Dominican",
    "Dutch", "East Timorese", "Ecuadorean", "Egyptian", "Emirati", "Equatorial Guinean", "Eritrean", "Estonian", "Eswatini",
    "Ethiopian", "Fijian", "Finnish", "French", "Gabonese", "Gambian", "Georgian", "German", "Ghanaian", "Greek",
    "Grenadian", "Guatemalan", "Guinean", "Guyanese", "Haitian", "Honduran", "Hungarian", "Icelandic", "Indian", "Indonesian",
    "Iranian", "Iraqi", "Irish", "Israeli", "Italian", "Ivorian", "Jamaican", "Japanese", "Jordanian", "Kazakh",
    "Kenyan", "Kiribati", "Kuwaiti", "Kyrgyz", "Lao", "Latvian", "Lebanese", "Lesotho", "Liberian", "Libyan",
    "Liechtensteiner", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malawian", "Malaysian", "Maldivian",
    "Malian", "Maltese", "Marshallese", "Mauritanian", "Mauritian", "Mexican", "Micronesian", "Moldovan", "Monacan",
    "Mongolian", "Montenegrin", "Moroccan", "Mozambican", "Myanmar", "Namibian", "Nauruan", "Nepalese", "New Zealander",
    "Nicaraguan", "Nigerian", "North Korean", "Norwegian", "Omani", "Pakistani", "Palauan", "Palestinian", "Panamanian",
    "Papua New Guinean", "Paraguayan", "Peruvian", "Philippine", "Polish", "Portuguese", "Qatari", "Romanian", "Russian",
    "Rwandan", "Saint Lucian", "Salvadoran", "Samoan", "Saudi", "Scottish", "Senegalese", "Serbian", "Seychellois",
    "Sierra Leonean", "Singaporean", "Slovak", "Slovenian", "Solomon Islander", "Somali", "South African", "South Korean",
    "South Sudanese", "Spanish", "Sri Lankan", "Sudanese", "Surinamese", "Swedish", "Swiss", "Syrian", "Taiwanese",
    "Tajik", "Tanzanian", "Thai", "Togolese", "Tongan", "Trinidadian", "Tunisian", "Turkish", "Turkmen", "Tuvaluan",
    "Ugandan", "Ukrainian", "Uruguayan", "Uzbek", "Venezuelan", "Vietnamese", "Welsh", "Yemeni", "Zambian", "Zimbabwean",

    # Historical nationalities
    "Austro-Hungarian", "Prussian", "Bohemian", "Ottoman", "Soviet", "Yugoslav", "Czechoslovak", "West German", "East German",
    "Rhodesian", "Serbo-Croatian", "Mandarin", "Manchu", "Ming", "Byzantine",

    # Stateless or refugee identities
    "Stateless", "Unknown",

    # Manually added for necessity
    "Yugoslavian", "USSR"
]

# Manual correction mapping for specific cases
manual_corrections = {
    "argentinian": "Argentine",
    "luxembourger": "Luxembourgish",
    "germna": "German",  # Common OCR mistake
    "polan": "Polish"  # Common OCR mistake
}

# Convert all nationalities in the DataFrame to lowercase and strip spaces
df["Nationality"] = df["Nationality"].str.lower().str.strip()

# Apply manual corrections first
df["Nationality"] = df["Nationality"].replace(manual_corrections)

# Fuzzy matching function for remaining uncorrected values
def fuzzy_correct(nationality):
    if pd.isna(nationality):  # Skip NaN values
        return None
    match, score = process.extractOne(nationality, valid_nationalities)
    return match if score > 85 else nationality  # Keep original if match 

# Apply fuzzy matching for remaining uncorrected values
df["Nationality"] = df["Nationality"].apply(fuzzy_correct)

df["Nationality"] = df["Nationality"].replace(["Stateless", "Unknown", "", " "], np.nan)

# Identify cases where fuzzy matching changed the nationality
df_fuzzy_changed = df[df["Nationality"] != df["Nationality"]]
print(df_fuzzy_changed['Nationality'].value_counts())

Series([], Name: count, dtype: int64)


In [31]:
# Find nationalities not in the valid list
valid_nationalities.append(None)
df_unrecognized = df[~df["Nationality"].isin(valid_nationalities)]
# Show the indexes and the unrecognized nationalities
df_nationality_cleaned = df[~df["Nationality"].isin(df_unrecognized['Nationality'])].reset_index(drop=True)

# Print the cleaned data
print("Cleaned Data without Unrecognized Nationalities:")
print(df_nationality_cleaned)

Cleaned Data without Unrecognized Nationalities:
      Birth Place Nationality Alternative Nationality 1  \
0           Other      Polish                       NaN   
1           Other        None                       NaN   
2           Other        None                       NaN   
3        Warschau      Polish                       NaN   
4           Other      German                       NaN   
...           ...         ...                       ...   
49396       Other        None                       NaN   
49397       Other        None                       NaN   
49398   Amsterdam        None                       NaN   
49399       Other    Canadian                 Hungarian   
49400       Other    American                       NaN   

      Alternative Nationality 2        Religion  \
0                           NaN  Roman Catholic   
1                           NaN             NaN   
2                           NaN             NaN   
3                           NaN       

In [34]:
# Define threshold (0.01% of total entries)
threshold = 0.001 * len(df)  # 0.01% of total dataset size

# Count occurrences of each nationality
nationality_counts = df['Nationality'].value_counts()

# Identify rare nationalities (those below the threshold)
rare_nationalities = nationality_counts[nationality_counts < threshold].index

# Replace rare nationalities with "Other"
df['Nationality'] = df['Nationality'].replace(rare_nationalities, 'Other')

# Display updated value counts
print(df['Nationality'].value_counts())


Nationality
Polish         6692
Israeli        6124
German         3088
Italian        1478
American       1253
Other           872
Hungarian       843
Czech           832
French          831
Yugoslavian     553
Romanian        522
Austrian        376
Russian         361
Ukrainian       156
Latvian         127
Canadian        124
USSR            114
Lithuanian      113
Dutch            85
Australian       77
British          65
Greek            65
Spanish          54
Name: count, dtype: int64


In [36]:
df.head()

df.to_csv("data/Power_BI_compatible_dataset.csv", index=False)  # Saves without index